# Docker Image Build & Deploy

Initialize local variables from the shared project variables file.

In [11]:
from project_vars import project_variables

env = project_variables['environment']
project_id = project_variables['project_id']
project_name = project_variables['project_name']
region = project_variables['region']

This notebook is often used standalone to build and deploy a docker image. So we want to authenticate to GCP again.

In [2]:
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=E6UoAqelFsT32pzuyuuj42Zn5gbjHK&access_type=offline&code_challenge=7lu34ITjVfW0cl2IVRfrtMV3NRICDVCyb10RRzcsqbc&code_challenge_method=S256


You are now logged in as [m.jarnot@yahoo.com].
Your current project is [fs-microservices].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [3]:
!gcloud config set project {project_id}

Updated property [core/project].


Configure GCP to use Docker.

In [4]:
!gcloud auth configure-docker


{
  "credHelpers": {
    "asia.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud",
    "us.gcr.io": "gcloud"
  }
}
Adding credentials for all GCR repositories.
gcloud credential helpers already registered correctly.


## Service Build & Deploy

You need to build and deploy as many docker images as you have services. You also need to update services array when services change in `project_vars.py` file. The following code block generates select field that you can use to choose what service you want to deploy.

In [37]:
import ipywidgets as widgets
from IPython.display import display

services = [
    {"name": "auth-service", "folder": "service-auth"},
    {"name": "events-service", "folder": "service-events"},
    {"name": "gateway-service", "folder": "service-gateway"},
    {"name": "storybook-service", "folder": "storybook"},
    {"name": "web-service", "folder": "web"}
]

# Create a dropdown widget
dropdown = widgets.Dropdown(
    options=[(f"{s['name']} ({s['folder']})", s) for s in services],
    description='Select a service:',
    style={'description_width': 'initial'}
)

# Create output widgets to display the selected values
output_name = widgets.HTML()
output_folder = widgets.HTML()

def on_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        selected = change['new']
        global service_name, service_folder
        service_name = selected['name']
        service_folder = selected['folder']
        output_name.value = f"<b>service_name</b> = \"{service_name}\""
        output_folder.value = f"<b>service_folder</b> = \"{service_folder}\""

# Attach the callback to the dropdown
dropdown.observe(on_change, names='value')

# Display the widgets
display(dropdown, output_name, output_folder)

Dropdown(description='Select a service:', options=(('auth-service (service-auth)', {'name': 'auth-service', 'f…

HTML(value='')

HTML(value='')

Build the Docker image using the Dockerfile located in the parent directory.

In [40]:
# print service_name, service_folder
print(f"Building Docker Image for {service_name} in /apps/{service_folder}")

Building Docker Image for web-service in /apps/web


In [34]:
!docker build --no-cache -t {region}-docker.pkg.dev/{project_id}/{env}-repo/{env}-{service_name}:latest -f ../apps/{service_folder}/Dockerfile ../

[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.1s (2/5)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.43kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 486B                                          0.0s
 => [internal] load metadata for docker.io/library/node:18.20.3-alpine3.1  0.1s
 => [internal] load metadata for docker.io/library/node:18.20.3            0.1s
 => [auth] library/node:pull token for registry-1.docker.io                0.0s
[+] Building 0.3s (3/5)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.43kB                                    

Push Docker Image to Artifact Registry

In [35]:
!docker push {region}-docker.pkg.dev/{project_id}/{env}-repo/{env}-{service_name}:latest

The push refers to repository [us-central1-docker.pkg.dev/fs-microservices/dev-repo/dev-gateway-service]

bb2670ff: Preparing 
78ba67bc: Preparing 
4ec0822d: Preparing 
8af962d6: Preparing 
f0573244: Preparing 
9c8c00e3: Preparing 
9b3ef1bf: Preparing 
a7d078a0: Preparing 
ccf8ba19: Preparing 
ce7ef8cf: Preparing 
f1fdab11: Preparing 
f0573244: Pushed   280.5MB/395.3kBlatest: digest: sha256:e1e758a2cd969c45796ab3be7068d60e1dac3a856bbdfe061895b74800b6ed55 size: 2824


Deploy to Google Cloud Run

In [41]:
!gcloud run deploy {env}-{service_name} --image {region}-docker.pkg.dev/{project_id}/{env}-repo/{env}-{service_name}:latest --platform managed --region={region} --allow-unauthenticated

Deploying container to Cloud Run service [dev-web-service] in project [fs-microservices] region [us-central1]
Deploying new service...                                                       
  . Creating Revision...                                                       
  . Routing traffic...                                                         
  . Setting IAM Policy...                                                      
  Deploying new service...                                                     



⠛ Deploying new service...                                                     



⠹ Deploying new service...                                                     



⠼ Deploying new service...                                                     



⠶ Deploying new service...                                                     



⠧ Deploying new service...                                                     



⠏ Deploying new service...                                              